# Runnables

`Runnables` are, abstractly, objects that implement the `.invoke()` method to perform some logic, then return the result. This can be calling an LLM, retrieving documents, formatting input or output, or even something custom. They can also override [other methods](/docs/expression_language/interface), such as `.stream()`, to support custom behavior for batching and streaming, but these will all delegate to this single `.invoke()` method by default.

It can be helpful to think of a `Runnable` as a protocol or interface rather than a concrete object - there are no concrete `Runnable` instances, only implementations. Most components in LangChain are runnables.

## Chaining

One important consequence of this shared single call is that it allows any two runnables to be "chained" together into sequences using the pipe operator (`|`), where the output of the previous runnable's `.invoke()` call is passed as input to the next runnable. The resulting `RunnableSequence` is itself a runnable, which means it can be invoked, streamed, or piped just like any other runnable.

For example, a common pattern in LangChain is using a prompt to format input into an LLM:

In [1]:
%pip install --upgrade --quiet langchain langchain-anthropic

You should consider upgrading via the '/Users/jacoblee/.pyenv/versions/3.10.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatAnthropic(model_name="claude-3-haiku-20240307")

chain = prompt | model

Prompts and models are both runnable, so we can chain them together. We can then invoke the resulting sequence like any other runnable:

In [3]:
chain.invoke({"topic": "bears"})

AIMessage(content="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nBa dum tss! Gotta love a good punny bear joke. Let me know if you want to hear another one!", response_metadata={'id': 'msg_01LegAaB9E9wE2QbNNpPXuym', 'content': [ContentBlock(text="Here's a bear joke for you:\n\nWhy don't bears wear socks? \nBecause they have bear feet!\n\nBa dum tss! Gotta love a good punny bear joke. Let me know if you want to hear another one!", type='text')], 'model': 'claude-3-haiku-20240307', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': Usage(input_tokens=13, output_tokens=57)})

We can even combine this chain with more runnables to create another chain. This may involve some input/output formatting using other types of runnables, depending on the required inputs and outputs of the chain components:

In [4]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = (
    chain | {"joke": StrOutputParser()} | analysis_prompt | model | StrOutputParser()
)

In [5]:
composed_chain.invoke({"topic": "bears"})

'Those were some pretty playful and punny bear jokes! I had a chuckle at the "bear feet" and "gummy bear" ones. Wordplay and puns can make for some lighthearted humor. While humor is subjective, I think those bear-themed jokes fit the silly and playful category quite well. Feel free to share any more bear-related puns or jokes if you have them - I\'m always up for a bit of good-natured silliness!'

## Streaming

While all runnables support the `.stream()` method, which returns a generator that yields output as it becomes available, by default by just returning the result of `.invoke()`, some runnables support multiple chunks. One case where this is useful is when calling LLMs and chat models as calls may be long-running.

When streamed, runnables in a chain will automatically aggregate output from previous steps when appropriate. In the above example, the prompt template requires the fully aggregate input from the previous chain to generate a complete input, so the sequence aggregates output from the chain before passing it to the prompt template. This interrupts streaming.

On the other hand, many output parsers and formatting components like [`RunnableParallel`](/docs/expression_language/primitives/map) and [`RunnablePassthrough`](/docs/expression_language/primitives/passthrough) are designed to handle individual chunks and transform them into some other output. This means that in the above composed chain, the final `StrOutputParser()` that converts chat model output into a string will not interrupt streaming. Let's see it in action:

In [7]:
stream = composed_chain.stream({"topic": "bears"})

for chunk in stream:
    print(chunk)

That
's
 a
 classic
 "
bear
 feet
"
 p
un
 joke
.
 P
uns
 can
 be
 quite
 hit
-
or
-
miss
 when
 it
 comes
 to
 humor
,
 as
 some
 people
 really
 enjoy
 them
 while
 others
 find
 them
 gr
oan
-
worthy
.
 Whether
 this
 particular
 bear
 s
ocks
 joke
 is
 funny
 or
 not
 is
 subj
ective
.
 I
'd
 say
 it
's
 a
 fairly
 straight
forward
,
 simple
 p
un
 that
 some
 people
 might
 find
 m
ild
ly
 amusing
,
 while
 others
 may
 just
 ch
uck
le
 or
 roll
 their
 eyes
 at
 the
 p
unch
line
.
 Humor
 is
 quite
 personal
,
 so
 the
 f
unn
iness
 of
 a
 joke
 really
 depends
 on
 the
 individual
's
 sense
 of
 humor
 and
 preferences
.


We see that we get many chunks, even though the streaming chat model wasn't the final step.

The rest of this section contains information about low-level utility primitives and concepts around runnables that are useful for formatting inputs and outputs at various steps. It may be more helpful to use it as a reference as needed rather than studying them all closely immediately.